In [ ]:
%load_ext autoreload
%autoreload 2

from pydrake.geometry import StartMeshcat

from iiwa_batter.sandbox.hydroelastic_tuning import run_hydroelastic_tuning
from iiwa_batter.assets.make_assets import write_assets
from iiwa_batter import CONTACT_DT, REALTIME_DT

In [ ]:
meshcat = StartMeshcat()

In [ ]:
# ENSURE THIS IS COPIED EXACTLY INTO make_assets.py !!!
bat_modulus = 1.2e12
ball_modulus = 6e9
mu_dynamic = 0.5
rigid_bat = False
write_assets(bat_modulus, ball_modulus, 1e-3, 1e-3, mu_dynamic, 0.014, rigid_bat)

In [ ]:
import numpy as np

# ENSURE THIS IS COPIED EXACTLY INTO make_assets.py !!!
bat_modulus = 1.2e12
ball_modulus = 6e9
mu_dynamic = 0.5
rigid_bat = False
dt = CONTACT_DT
#dt = CONTACT_DT * 20
#write_assets(bat_modulus, ball_modulus, 1e-3, 1e-3, mu_dynamic, 0.014, rigid_bat)

velocities = np.linspace(20, 60, 9)
#velocities = np.linspace(10, 110, 21)

velocity_results = {}
for velocity in velocities:
    cor, ball_x_positions, ball_x_velocities, times = run_hydroelastic_tuning(None, [1, 0, 0], -1*velocity, dt=dt, record_time=0.2, debug_plot=False)
    velocity_results[velocity] = {
        "cor": cor,
        "ball_x_positions": ball_x_positions,
        "ball_x_velocities": ball_x_velocities,
        "times": times,
    }


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 1, figsize=(6, 6), sharex=True)

TICK_FONTSIZE = 14
LABEL_FONTSIZE = 16
TITLE_FONTSIZE = 18

for velocity, result in velocity_results.items():
    ball_x_positions = result["ball_x_positions"]
    ball_x_velocities = result["ball_x_velocities"]
    times = result["times"]
    axes[0].plot(times, ball_x_positions, label=f"velocity: {velocity}")
    axes[1].plot(times, ball_x_velocities, label=f"velocity: {velocity}")

axes[0].set_ylabel("Ball x position [m]", color="white", fontsize=LABEL_FONTSIZE)
#axes[0].set_xlabel("Time [s]", color="white", fontsize=LABEL_FONTSIZE)
axes[0].grid(True)
axes[0].set_ylim([0, 1])
axes[1].set_ylabel("Ball x velocity [m/s]", color="white", fontsize=LABEL_FONTSIZE)
axes[1].set_xlabel("Time [s]", color="white", fontsize=LABEL_FONTSIZE)
axes[1].grid(True)

for ax in axes:
    ax.tick_params(axis='x', colors='white', labelsize=TICK_FONTSIZE)
    ax.tick_params(axis='y', colors='white', labelsize=TICK_FONTSIZE)
    ax.set_xlim([0, 0.12])
    ax.set_facecolor("#1F1F1F")


#axes[0].legend()

axes[0].set_title(f"Contact simulation with dt {dt:.1e} s", color="white", fontsize=TITLE_FONTSIZE)

fig.patch.set_facecolor("#000000")

fig.tight_layout()

fig.savefig(f"notebook_images/hydroelastic_tuning/ball_trajectory_hydroelastic_tuning_dt_{dt:.0e}.png")


In [ ]:
# Plot the coefficient of restitution for each velocity
velocities = np.array(list(velocity_results.keys()))
cors = np.array([result["cor"] for result in velocity_results.values()])

fig, ax = plt.subplots()
ax.plot(velocities, cors, marker="o", label="Simulated CCoR", color="cyan")
ax.set_xlabel("Pitch Speed [m/s]", fontsize=LABEL_FONTSIZE, color="white")
ax.set_ylabel("Cylindrical Coefficient of Restitution", fontsize=LABEL_FONTSIZE, color="white")
ax.grid(True)
# Put a vertical line at 26.8224 m/s
# Put a horizontal line at 0.546
ax.axhline(0.546, color="yellow", linestyle="--", label="MLB Regulation CCoR")
ax.axvline(26.8224, color="red", linestyle="--", label="Test Speed (60 mph)")
ax.set_title("Hydroelastic Contact Simulation vs Reality", fontsize=TITLE_FONTSIZE, color="white")
legend = ax.legend()
legend.get_frame().set_facecolor("#1F1F1F")
ax.set_ylim([0.42, 0.6])
for text in legend.get_texts():
    text.set_color("white")
    text.set_fontsize(TICK_FONTSIZE)

ax.tick_params(axis='x', colors='white', labelsize=TICK_FONTSIZE)
ax.tick_params(axis='y', colors='white', labelsize=TICK_FONTSIZE)
ax.set_facecolor("#1F1F1F")
fig.patch.set_facecolor("#0F0F0F")
fig.tight_layout()

fig.savefig(f"notebook_images/hydroelastic_tuning/ccor_vs_velocity_hydroelastic_tuning_dt_{dt:.0e}.png")